In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [2]:
import cvxpy as cp
import numpy as np

In [3]:
from solara.constants import PROJECT_PATH
import solara.environment.photovoltaics
import solara.environment.loads
import solara.environment.grids
pv_data_path = PROJECT_PATH + "/data/solar_trace_data/PV_5796.txt"
load_data_path = PROJECT_PATH + "/data/solar_trace_data/load_5796.txt"

pv_model = solara.environment.photovoltaics.DataPV(data_path=pv_data_path)
load_model = solara.environment.loads.DataLoad(data_path=load_data_path)
grid_model = solara.environment.grids.PeakGrid()

In [4]:
start = np.random.randint(len(pv_model.data) // 24) * 24

pv_model.reset(start=start)
load_model.reset(start=start)
print("Starting point: ", pv_model.start)
for i in range(24):
    load = load_model.get_next_load()
    pv_gen = pv_model.get_next_generation()
    net_load = np.maximum(0,load-pv_gen)
    price = grid_model.draw_power(net_load)
    print("Load: {:.3f}, PV generation: {:.3f}, Net load: {:.3f}, Price: {:.3f}".format(load, pv_gen, net_load, price))

Starting point:  8688
Load: 0.386, PV generation: 0.000, Net load: 0.386, Price: 0.054
Load: 0.370, PV generation: 0.000, Net load: 0.370, Price: 0.052
Load: 0.379, PV generation: 0.000, Net load: 0.379, Price: 0.053
Load: 0.373, PV generation: 0.000, Net load: 0.373, Price: 0.052
Load: 0.402, PV generation: 0.000, Net load: 0.402, Price: 0.056
Load: 0.461, PV generation: 0.000, Net load: 0.461, Price: 0.065
Load: 0.792, PV generation: 0.000, Net load: 0.792, Price: 0.111
Load: 0.363, PV generation: 0.003, Net load: 0.360, Price: 0.050
Load: 0.261, PV generation: 0.096, Net load: 0.165, Price: 0.023
Load: 0.252, PV generation: 0.344, Net load: 0.000, Price: 0.000
Load: 0.288, PV generation: 0.711, Net load: 0.000, Price: 0.000
Load: 0.276, PV generation: 0.791, Net load: 0.000, Price: 0.000
Load: 0.291, PV generation: 0.791, Net load: 0.000, Price: 0.000
Load: 0.273, PV generation: 0.786, Net load: 0.000, Price: 0.000
Load: 0.262, PV generation: 0.562, Net load: 0.000, Price: 0.000
Loa

In [5]:
import solara.environment.battery_control
import solara.environment.batteries

battery_model = solara.environment.batteries.LithiumIonBattery(20, "NMC", 1/10.0)
pv_model = solara.environment.photovoltaics.DataPV(data_path=pv_data_path)
load_model = solara.environment.loads.DataLoad(data_path=load_data_path)
grid_model = solara.environment.grids.PeakGrid()

env = solara.environment.battery_control.BatteryControlEnv(battery=battery_model,
        pv_system = pv_model,
        grid = grid_model,
        load = load_model,)

In [6]:
done = False
env.reset()
for i in range(26):
    if not done:
        obs, _, done, _ = env.step(np.array([0.3]))
        print(i, obs)
    else:
        print("Episode done.")

0 [0.603717 0.       1.9884   1.       0.603717 0.      ]
1 [0.60675  0.       3.9768   2.       1.210467 0.      ]
2 [0.854733 0.       5.9652   3.       2.0652   0.      ]
3 [0.839817 0.       7.9536   4.       2.905017 0.      ]
4 [0.8112   0.       9.942    5.       3.716217 0.      ]
5 [ 0.851333  0.       11.9304    6.        4.56755   0.      ]
6 [4.175830e-01 6.452000e-03 1.391880e+01 7.000000e+00 4.985133e+00
 6.452000e-03]
7 [ 1.395383  0.040174 15.9072    8.        6.380516  0.046626]
8 [ 1.636567    0.105383   17.68990345  9.          8.017083    0.152009  ]
9 [ 1.4284      0.197368   18.64982069 10.          9.445483    0.349377  ]
10 [ 0.515033    0.191018   19.19834483 11.          9.960516    0.540395  ]
11 [ 0.296183    0.07662    19.54117241 12.         10.256699    0.617015  ]
12 [ 0.441983    0.159432   19.67830345 13.         10.698682    0.776447  ]
13 [ 0.5395      0.146029   19.81543448 14.         11.238182    0.922476  ]
14 [ 0.615283  0.136064 19.884    15.  